In [1]:
import warnings
warnings.filterwarnings('ignore')
from langchain_google_genai import GoogleGenerativeAI
google_api_key = "Enter_your_key"
llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key = google_api_key, temperature = 0.2)

In [2]:
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db_user = "root"
db_password = "enter_passwd"
db_host = "localhost"
db_name = "books_database"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE books (
	book_id INTEGER NOT NULL AUTO_INCREMENT, 
	publisher ENUM('Print House','Reader Series','Rainbook','Glassowl','Wooden Letters') NOT NULL, 
	genre ENUM('Fiction','Mystery','Thriller','Fantasy','Biography') NOT NULL, 
	lang ENUM('English','Russian','Japanese','German','Chinese') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (book_id), 
	CONSTRAINT books_chk_1 CHECK ((`price` between 10 and 50))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from books table:
book_id	publisher	genre	lang	price	stock_quantity
1	Rainbook	Biography	German	49	69
2	Rainbook	Fiction	English	15	55
3	Print House	Fiction	Chinese	21	78
*/


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	book_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(book_id) REFERENCES books (book_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0

In [4]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("How many fantasy books do we have by Rainbook in Chinese?")



> Entering new SQLDatabaseChain chain...
How many fantasy books do we have by Rainbook in Chinese?
SQLQuery:SELECT count(*) FROM books WHERE publisher = 'Rainbook' AND genre = 'Fantasy' AND lang = 'Chinese'
SQLResult: [(1,)]
Answer:1
> Finished chain.


In [7]:
qns1 = db_chain.run("SELECT stock_quantity FROM books WHERE publisher = 'Rainbook' AND genre = 'Fantasy' AND lang = 'Chinese'")



> Entering new SQLDatabaseChain chain...
SELECT stock_quantity FROM books WHERE publisher = 'Rainbook' AND genre = 'Fantasy' AND lang = 'Chinese'
SQLQuery:SELECT stock_quantity FROM books WHERE publisher = 'Rainbook' AND genre = 'Fantasy' AND lang = 'Chinese'
SQLResult: [(58,)]
Answer:58
> Finished chain.


In [8]:
qns2 = db_chain("What is the total number of books in Russian?")



> Entering new SQLDatabaseChain chain...
What is the total number of books in Russian?
SQLQuery:SELECT count(*) FROM books WHERE lang = 'Russian'
SQLResult: [(11,)]
Answer:11
> Finished chain.


In [9]:
qns2 = db_chain.run("SELECT SUM(stock_quantity) FROM books WHERE lang = 'Russian'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(stock_quantity) FROM books WHERE lang = 'Russian'
SQLQuery:SELECT SUM(stock_quantity) FROM books WHERE lang = 'Russian'
SQLResult: [(Decimal('623'),)]
Answer:623
> Finished chain.


In [ ]:
qns3 = db_chain.run("If we have to sell all the Glassowl books today with discounts applied. How much revenue our store will generate (post discounts)?")

In [10]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, book_id from books where publisher = 'Glassowl'
group by book_id) a left join discounts on a.book_id = discounts.book_id
 """

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, book_id from books where publisher = 'Glassowl'
group by book_id) a left join discounts on a.book_id = discounts.book_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, book_id from books where publisher = 'Glassowl'
group by book_id) a left join discounts on a.book_id = discounts.book_id
SQLResult: [(Decimal('28584.950000'),)]
Answer:28584.95
> Finished chain.


In [11]:
qns4 = db_chain.run("If we sell them without discount, how much revenue will be generated on books by Reader Series?")



> Entering new SQLDatabaseChain chain...
If we sell them without discount, how much revenue will be generated on books by Reader Series?
SQLQuery:SELECT SUM(price) FROM books WHERE publisher = 'Reader Series'
SQLResult: [(Decimal('369'),)]
Answer:369
> Finished chain.


In [12]:
sql_code2 = """
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE publisher = 'Reader Series'
"""

qns4 = db_chain.run(sql_code2)



> Entering new SQLDatabaseChain chain...

SELECT SUM(price * stock_quantity) FROM t_shirts WHERE publisher = 'Reader Series'

SQLQuery:SELECT SUM(price * stock_quantity) FROM books WHERE publisher = 'Reader Series'
SQLResult: [(Decimal('20067'),)]
Answer:20067
> Finished chain.


In [ ]:
qns5 = db_chain.run("What publisher do we have the least stock of?")

In [14]:
sql_code3 = "SELECT publisher FROM books group by publisher order by sum(stock_quantity) asc limit 1"
qns5 = db_chain.run(sql_code3)



> Entering new SQLDatabaseChain chain...
SELECT publisher FROM books group by publisher order by sum(stock_quantity) asc limit 1
SQLQuery:SELECT publisher FROM books group by publisher order by sum(stock_quantity) asc limit 1
SQLResult: [('Wooden Letters',)]
Answer:Wooden Letters
> Finished chain.


In [15]:
qns6 = db_chain.run("SELECT SUM(stock_quantity) FROM books where lang = 'Chinese'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(stock_quantity) FROM books where lang = 'Chinese'
SQLQuery:SELECT SUM(stock_quantity) FROM books where lang = 'Chinese'
SQLResult: [(Decimal('690'),)]
Answer:690
> Finished chain.


#### Few Shot Learning

In [16]:
few_shots = [
    {'Question' : "How many fantasy books do we have by Rainbook in Chinese?",
     'SQLQuery' : "SELECT stock_quantity FROM books WHERE publisher = 'Rainbook' AND genre = 'Fantasy' AND lang = 'Chinese'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},
    {'Question' : "What is the total number of books in Russian?",
     'SQLQuery' : "SELECT SUM(stock_quantity) FROM books WHERE lang = 'Russian'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns2},
    {'Question': "If we have to sell all the Glassowl books today with discounts applied. How much revenue our store will generate (post discounts)?",
     'SQLQuery':"select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, book_id from books where publisher = 'Glassowl' group by book_id) a left join discounts on a.book_id = discounts.book_id'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns3},
    {'Question': "If we sell them without discount, how much revenue will be generated on books by Reader Series??" ,
     'SQLQuery' : "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE publisher = 'Reader Series'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns4},
    {'Question': "What publisher do we have the least stock of?",
     'SQLQuery' : "SELECT publisher FROM books group by publisher order by sum(stock_quantity) asc limit 1",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5},
    {'Question': "How many Chinese books do we have?",
     'SQLQuery' : "SELECT SUM(stock_quantity) FROM books where lang = 'Chinese'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns6
     }
]

### Creating Semantic Similarity based Example Sector

In [18]:
from langchain_core.example_selectors.semantic_similarity import SemanticSimilarityExampleSelector
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.chroma import Chroma


embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

to_vectorize = [" ".join(example.values()) for example in few_shots]
to_vectorize

["How many fantasy books do we have by Rainbook in Chinese? SELECT stock_quantity FROM books WHERE publisher = 'Rainbook' AND genre = 'Fantasy' AND lang = 'Chinese' Result of the SQL query 58",
 "What is the total number of books in Russian? SELECT SUM(stock_quantity) FROM books WHERE lang = 'Russian' Result of the SQL query 623",
 "If we have to sell all the Glassowl books today with discounts applied. How much revenue our store will generate (post discounts)? select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, book_id from books where publisher = 'Glassowl' group by book_id) a left join discounts on a.book_id = discounts.book_id' Result of the SQL query 28584.95",
 "If we sell them without discount, how much revenue will be generated on books by Reader Series?? SELECT SUM(price * stock_quantity) FROM t_shirts WHERE publisher = 'Reader Series' Result of the SQL query 20067",
 'What publish

In [19]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [20]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

In [21]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [22]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [23]:
from langchain_core.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [24]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [25]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [26]:
new_chain.run("Is there any discount on any of the English books?")



> Entering new SQLDatabaseChain chain...
Is there any discount on any of the English books?
SQLQuery:SELECT count(*) FROM discounts WHERE book_id IN (SELECT book_id FROM books WHERE lang = 'English')
SQLResult: [(3,)]
Answer:Yes
> Finished chain.


'Yes'

In [27]:
new_chain.run("What genre do we have the maximum stock of and how much?")



> Entering new SQLDatabaseChain chain...
What genre do we have the maximum stock of and how much?
SQLQuery:SELECT genre, MAX(stock_quantity) FROM books GROUP BY genre ORDER BY MAX(stock_quantity) DESC LIMIT 1
SQLResult: [('Fantasy', 100)]
Answer:Fantasy, 100
> Finished chain.


'Fantasy, 100'

In [28]:
new_chain.run("If we have to sell all the Glassowl books today with discounts applied. How much revenue our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Glassowl books today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, book_id from books where publisher = 'Glassowl' group by book_id) a left join discounts on a.book_id = discounts.book_id
SQLResult: [(Decimal('28584.950000'),)]
Answer:28584.95
> Finished chain.


'28584.95'

In [29]:
new_chain.run("How many Japanese books do we have?") #still producing the wrong answer



> Entering new SQLDatabaseChain chain...
How many Japanese books do we have?
SQLQuery:SELECT COUNT(*) FROM books WHERE lang = 'Japanese'
SQLResult: [(14,)]
Answer:14
> Finished chain.


'14'

In [30]:
new_chain.run("How many Japanese books do we have in stock?") 

#needs the word "stock" included to get correct answer
#even with stock gives wrong answer sometimes



> Entering new SQLDatabaseChain chain...
How many Japanese books do we have in stock?
SQLQuery:SELECT SUM(stock_quantity) FROM books WHERE lang = 'Japanese'
SQLResult: [(Decimal('854'),)]
Answer:854
> Finished chain.


'854'

In [31]:
new_chain.run("How many Japanese books do we have left in total?") 

#this one gives the right answer always



> Entering new SQLDatabaseChain chain...
How many Japanese books do we have left in total?
SQLQuery:SELECT SUM(stock_quantity) FROM books WHERE lang = 'Japanese'
SQLResult: [(Decimal('854'),)]
Answer:854
> Finished chain.


'854'

In [35]:
new_chain.run("How many romance book do we have left in total?") 
#is equating None to 100 for mysterious reasons



> Entering new SQLDatabaseChain chain...
How many romance book do we have left in total?
SQLQuery:SELECT sum(stock_quantity) FROM books WHERE genre = 'Romance'
SQLResult: [(None,)]
Answer:100
> Finished chain.


'100'